In [1]:
import pandas as pd
import altair as alt

In [35]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [11]:
dylan = pd.read_csv('dylan.csv')[1:]
marley = pd.read_csv('marley.csv')[1:]
springsteen = pd.read_csv('springsteen.csv')[1:]
winehouse = pd.read_csv('winehouse.csv')[1:]

In [19]:
dylan['date'] = pd.date_range(start='2004-01-01', periods=len(dylan), freq='MS')
marley['date'] = pd.date_range(start='2004-01-01', periods=len(marley), freq='MS')
springsteen['date'] = pd.date_range(start='2004-01-01', periods=len(springsteen), freq='MS')
winehouse['date'] = pd.date_range(start='2004-01-01', periods=len(winehouse), freq='MS')

In [80]:
release_dates = pd.DataFrame({
    'title': ['A Complete Unknown', 'One Love', 'Deliver Me from Nowhere', 'Back to Black'],
    'artist': ['Bob Dylan', 'Bob Marley', 'Bruce Springsteen', 'Amy Winehouse'],
    'release_month': ['2024-12-01', '2024-02-01', '2025-10-01', '2024-04-01']
})

In [81]:
dylan['artist'] = 'Bob Dylan'
marley['artist'] = 'Bob Marley'
springsteen['artist'] = 'Bruce Springsteen'
winehouse['artist'] = 'Amy Winehouse'

searches = pd.concat([dylan, marley, springsteen, winehouse], ignore_index=True)

In [82]:
searches = searches.merge(release_dates, on='artist', how='left')

In [83]:
searches['searches'] = pd.to_numeric(searches['Category: All categories'], errors='coerce')
searches['artist_title'] = searches['artist'] + ': "' + searches['title'] + '"'

In [84]:
searches_filt = searches[searches['date'] >= '2020-01-01'].reset_index(drop=True)

In [101]:
searches_filt['artist_title'].unique()

array(['Bob Dylan: "A Complete Unknown"', 'Bob Marley: "One Love"',
       'Bruce Springsteen: "Deliver Me from Nowhere"',
       'Amy Winehouse: "Back to Black"'], dtype=object)

In [154]:
colors = {
    'Bob Dylan: "A Complete Unknown"': '#e6224b',
    'Bob Marley: "One Love"': '#36b7b4', 
    'Bruce Springsteen: "Deliver Me from Nowhere"': '#f4c245',
    'Amy Winehouse: "Back to Black"': '#179fdb'
}

chart_names = {
    'Bob Dylan: "A Complete Unknown"': 'dylan_chart',
    'Bob Marley: "One Love"': 'marley_chart',
    'Bruce Springsteen: "Deliver Me from Nowhere"': 'springsteen_chart',
    'Amy Winehouse: "Back to Black"': 'winehouse_chart'
}

for i in ['Bob Dylan: "A Complete Unknown"', 'Bob Marley: "One Love"', 'Bruce Springsteen: "Deliver Me from Nowhere"', 'Amy Winehouse: "Back to Black"']:
    subset = searches_filt[searches_filt['artist_title'] == i]
    
    # Only show x-axis title for Springsteen chart
    x_title = 'Note: Google search interest is calculated relative to the highest point on the chart for the given region and time.' if i == 'Bruce Springsteen: "Deliver Me from Nowhere"' else ''

    lines = alt.Chart(subset).mark_line(color=colors[i]).encode(
        x=alt.X('date:T', axis=alt.Axis(tickCount=5, 
                                        labelFontSize=14,
                                        titleX=0,
                                        titleY=25,
                                        titleAnchor='start',
                                        title=x_title,
                                        titleFontSize=12)),
        y=alt.Y('searches:Q', axis=alt.Axis(labelFontSize=14, title = 'Google search interest', titleFontSize=14, titleX= -21, titleY= -10)),
    ).properties(
        width=600,
        height=400,
        title=alt.TitleParams(text=f'{i} ', fontSize=16, anchor='middle', dy=0)
    )

    releases = alt.Chart(subset).mark_rule(color='grey', strokeDash=[4,4]).encode(
        x='release_month:T',
        tooltip=['artist:N', 'title:N', 'release_month:T']
    )

    text = alt.Chart(subset).mark_text(align='left', dx=-140, dy=-190, fontSize=14).encode(
        text=alt.value('Biopic release month'),
        x='release_month:T',
        tooltip=['artist:N', 'title:N', 'release_month:T'],

    )

    chart = releases + lines + text
    
    # Save to variable based on artist
    if 'Dylan' in i:
        dylan_chart = chart
    elif 'Marley' in i:
        marley_chart = chart
    elif 'Springsteen' in i:
        springsteen_chart = chart
    elif 'Winehouse' in i:
        winehouse_chart = chart
    
    chart.show()

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

In [155]:
panels = alt.vconcat(
    alt.hconcat(dylan_chart, marley_chart),
    alt.hconcat(springsteen_chart, winehouse_chart)
).properties(
    title =alt.TitleParams(text='The Biopic Effect: How Biopics Influence Google Search Interest', fontSize=22, anchor='start', dx=-15, dy=-5),
)

panels

alt.VConcatChart(...)

In [156]:
# Save to png
panels.save('biopic_effect.png', scale_factor=2)
# Save to json
panels.save('biopic_effect.json', scale_factor=2)